In [2]:
import cv2 as cv
import numpy as np

In [4]:
"""
Simple iterative Lucas–Kanade for ONE point using a square window.
Uses getRectSubPix for subpixel sampling.
Returns (flow, ok, info_dict)
The core purpose of this function is to calculate the motion vector (flow) of a single feature point (pt) between two consecutive grayscale images:
prev_gray and next_gray.
"""

def lk_by_hand_iterative(prev_gray, next_gray, pt, win=21, iters=10, eps=1e-3, min_eig=1e-4):
    pt = np.array(pt, dtype=np.float32)  # (x, y)
    flow = np.zeros(2, dtype=np.float32)

    # Work in float
    prev_f = prev_gray.astype(np.float32)
    next_f = next_gray.astype(np.float32)

    for k in range(iters):
        # Patch from prev at fixed location pt
        patch1 = cv.getRectSubPix(prev_f, (win, win), tuple(pt))
        # Patch from next at warped location pt + flow
        patch2 = cv.getRectSubPix(next_f, (win, win), tuple(pt + flow))

        # Spatial gradients from patch1
        Ix = cv.Sobel(patch1, cv.CV_32F, 1, 0, ksize=3)
        Iy = cv.Sobel(patch1, cv.CV_32F, 0, 1, ksize=3)
        It = patch2 - patch1

        A = np.stack([Ix.reshape(-1), Iy.reshape(-1)], axis=1)  # (N,2)
        b = -It.reshape(-1)                                     # (N,)

        G = A.T @ A  # 2x2
        eigvals = np.linalg.eigvalsh(G)
        if eigvals[0] < min_eig:
            return flow, False, {"reason": "weak/edge-like feature (min eigen too small)", "eigvals": eigvals}

        # Least squares solve: A * delta = b
        delta, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
        delta = delta.astype(np.float32)

        flow += delta

        if float(np.linalg.norm(delta)) < eps:
            return flow, True, {"iters": k + 1, "eigvals": eigvals}

    return flow, True, {"iters": iters, "eigvals": eigvals}

In [5]:
# Perform a visual and computational comparison between two implementations of the Lucas-Kanade (LK) Optical Flow algorithm

def main():
    cap = cv.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("Cannot open webcam")

    print("Instructions:")
    print("- Press SPACE to capture frame A")
    print("- Press SPACE again to capture frame B (move camera/object slightly between captures)")
    print("- Then we pick a corner and compare OpenCV PyrLK vs by-hand LK for that point")
    print("- Press ESC to quit")

    frameA = None
    frameB = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        vis = frame.copy()
        cv.putText(vis, "SPACE: capture A/B | ESC: quit", (10, 25),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv.imshow("Capture", vis)

        key = cv.waitKey(1) & 0xFF
        if key == 27:
            cap.release()
            cv.destroyAllWindows()
            return
        if key == 32:  # SPACE
            if frameA is None:
                frameA = frame.copy()
                print("Captured frame A")
            elif frameB is None:
                frameB = frame.copy()
                print("Captured frame B")
                break

    cap.release()
    cv.destroyAllWindows()

    prev_gray = cv.cvtColor(frameA, cv.COLOR_BGR2GRAY)
    next_gray = cv.cvtColor(frameB, cv.COLOR_BGR2GRAY)

    # Detect corners in A
    p0 = cv.goodFeaturesToTrack(prev_gray, maxCorners=200, qualityLevel=0.01, minDistance=7, blockSize=7)
    if p0 is None:
        raise RuntimeError("No corners found. Try better lighting/texture.")

    # Choose the strongest corner (first one returned is usually strong)
    pt0 = p0[0, 0].astype(np.float32)  # (x, y)
    print(f"Chosen point (x,y): {pt0}")

    # OpenCV LK (single level for closer comparison)
    lk_params = dict(
        winSize=(21, 21),
        maxLevel=0,
        criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 30, 0.01),
        flags=0,
        minEigThreshold=1e-4
    )

    p1, st, err = cv.calcOpticalFlowPyrLK(prev_gray, next_gray, p0[:1].astype(np.float32), None, **lk_params)
    st = int(st[0, 0])
    if st != 1:
        print("OpenCV LK failed on this point. Try again with different motion/lighting.")
        return

    pt1 = p1[0, 0]
    flow_cv = pt1 - pt0

    # By-hand LK
    flow_hand, ok, info = lk_by_hand_iterative(prev_gray, next_gray, pt0, win=21, iters=10, eps=1e-3, min_eig=1e-4)

    print("\nResults:")
    print(f"OpenCV flow (u,v): {flow_cv}")
    print(f"By-hand flow (u,v): {flow_hand} | ok={ok} | info={info}")
    print(f"Difference (hand - cv): {flow_hand - flow_cv}")

In [7]:
main()

Instructions:
- Press SPACE to capture frame A
- Press SPACE again to capture frame B (move camera/object slightly between captures)
- Then we pick a corner and compare OpenCV PyrLK vs by-hand LK for that point
- Press ESC to quit
Captured frame A
Captured frame B
Chosen point (x,y): [ 97. 451.]

Results:
OpenCV flow (u,v): [27.232407 17.940643]
By-hand flow (u,v): [1.2539729 2.6567311] | ok=True | info={'iters': 10, 'eigvals': array([6906504.5, 7935297.5], dtype=float32)}
Difference (hand - cv): [-25.978434 -15.283913]
